In [10]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
#import libraries
%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("&lt;style>.container { width:98% !important; }&lt;/style>"))
nlp = spacy.load("en_core_web_sm")

In [3]:
#define function to clean text
def textclean(Text):
    Text = Text.str.replace('http\S+|www.\S+','', case=False)
    Text = Text.str.replace("(<br/>)", "")
    Text = Text.str.replace('(<a).*(>).*(</a>)', '')
    Text = Text.str.replace('(&amp)', '')
    Text = Text.str.replace('(&gt)', '')
    Text = Text.str.replace('(&lt)', '')
    Text = Text.str.replace('(\xa0)', ' ') 
    Text = Text.str.replace('[^A-Za-z ]+', ' ')
    Text = Text.str.replace('[^A-Za-z ]+', '')
    Text = Text.str.replace('—', '')
    Text = Text.str.replace('twitter|com|pic', '')
    return Text

In [4]:
#read in tweets and clean text
df_tweets_djt = pd.read_csv("Data/trump_tweets.csv") 
df_tweets_djt = df_tweets_djt[~df_tweets_djt['text'].str.contains("RT @")]
df_tweets_djt['text'] = textclean(df_tweets_djt['text'])
df_tweets_djt.rename(columns={'text':'Text'}, inplace=True)
df_tweets_djt = df_tweets_djt.drop(columns=['created_at', 'retweet_count', 'favorite_count','is_retweet','id_str','source'])
#add column to note the author of the tweet
df_tweets_djt['Author'] = 'Trump'
df_tweets_djt

,Text,Author
0,LOUISIANA Tomorrow you will head to the poll...,Trump
1,Louisiana REPUBLICANS thank you for a great ev...,Trump
2,I never saw so many subpoenas ShannonBream ...,Trump
3,So funny to watch Steve Kerr grovel and pander...,Trump
4,WHERE S HUNTER,Trump
...,...,...
15248,Well the year has officially begun I have man...,Trump
15249,sprinklermanus CNN realDonaldTrump they re...,Trump
15250,jallenaip Hillary said she was in a Fog of W...,Trump
15251,Happy New Year from MarALago Thank you to my...,Trump


In [5]:
#read in obama tweets and clean the text
df_tweets_dho = pd.read_csv("Data/obama_tweets.csv") 
df_tweets_dho = df_tweets_dho[~df_tweets_dho['Text'].str.contains("RT @")]
df_tweets_dho['Text'] = textclean(df_tweets_dho['Text'])
df_tweets_dho = df_tweets_dho.drop(columns=['Date', 'Retweets', 'Tweet ID','Favorites'])
#add column to note the author of the tweet
df_tweets_dho['Author'] = 'Obama'
df_tweets_dho

,Text,Author
0,My heart goes out to the victims and their fam...,Obama
1,Chuck Berry rolled over everyone who came befo...,Obama
2,On International Women s Day MichelleObama a...,Obama
3,Humbled to be recognized by a family with a le...,Obama
4,We asked You answered,Obama
...,...,...
6729,reasons that people are voting for Preside...,Obama
6730,At the final rally of his final campaign last ...,Obama
6731,It s Election Day This is your last chance to...,Obama
6732,Election Day is here Confirm your polling pla...,Obama


In [6]:
#combine tweets into 1 dataframe
df_combine = df_tweets_djt.append(df_tweets_dho,ignore_index=True)
df_combine

,Text,Author
0,LOUISIANA Tomorrow you will head to the poll...,Trump
1,Louisiana REPUBLICANS thank you for a great ev...,Trump
2,I never saw so many subpoenas ShannonBream ...,Trump
3,So funny to watch Steve Kerr grovel and pander...,Trump
4,WHERE S HUNTER,Trump
...,...,...
19356,reasons that people are voting for Preside...,Obama
19357,At the final rally of his final campaign last ...,Obama
19358,It s Election Day This is your last chance to...,Obama
19359,Election Day is here Confirm your polling pla...,Obama


In [7]:
#parse the next in spacy and put it into a new column
df_combine['Parsed'] = df_combine.Text.apply(nlp)

In [8]:
#use scatter text to build a corpus of words
corpus = st.CorpusFromParsedDocuments(df_combine, category_col='Author', parsed_col='Parsed').build()

In [9]:
#plot scatter text. Note this can take up to 5 minutes. 
html = produce_scattertext_explorer(corpus,
                                    category='Obama',
                                    category_name='Obama',
                                    not_category_name='Trump',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    show_characteristic=False,
                                    term_significance = st.LogOddsRatioUninformativeDirichletPrior())
                                    #metadata=df_combine['Text'])
file_name = 'Data/Obama_Trump.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)


C:\ProgramData\Anaconda3\lib\site-packages\scattertext\TermDocMatrixWithoutCategories.py:154: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  corpus_unigram_freq = corpus_freq_df.ix[[term for term
